In [ ]:

import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from torchvision import datasets
#import torchvision.transforms as transforms
from torch.utils.data import Dataset

from tqdm import tqdm
import torch.nn as nn
import random
import torch.nn.functional as F

import matplotlib.patches as mpatches
from models.modelsList import *



In [ ]:
print(torch.cuda.is_available())
torch.cuda.empty_cache()

# data upload & checks

In [ ]:

from qutip import fidelity, Qobj
from models.utils import *


1. data upload

In [ ]:
foldertype= 'A/depotsts/'
url ='data/fromMeasurementSettings/'+foldertype
#url = 'data/'data=pd.read_csv(url+'train_org.csv')
data=pd.read_csv(url+'train_org.csv')
data_noise=pd.read_csv(url+'train_noise.csv')
testorg = pd.read_csv(url+'depo02test_org.csv')
testnoise = pd.read_csv(url+'depo02test_noise.csv')

2. splitting

In [ ]:
# first off, I split the training datasets into train and validation
split = 30000
trainA, valA = data[1:split], data[split:39999] #I start form row 1 to be sure to avoid the panda header
trainNoise, valNoise = data_noise[1:split],data_noise[split:-1]
print(trainA.shape),print(valA.shape),print(trainNoise.shape),print(valNoise.shape)
del(data)
del(data_noise)


# datasets

In [ ]:
trainset=attentionDataset(trainA, trainNoise)
valset = attentionDataset(valA, valNoise)

testset=attentionDataset(testorg[:10], testnoise[:10])

del(testorg)
del(testnoise)


torch.manual_seed(32)

## ATTENTION BASED MODEL initalization

In [ ]:

batch_size= 200
epochs=30
#VECCIO VALORE DI EPOCHS 30
lr = 0.0001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:

model = Netbeta(256,3,1,256)


In [ ]:

#model=AutoEncoder(myenc,mydec).to(device)
model = model.to(device)
#print(model)
#summary(model, input_size=(1,16,16))

optimizer=torch.optim.Adam(model.parameters(), lr= lr)
#just a MSE
theloss = donatoLoss()


### dataloaders

In [ ]:
train_dataloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size,
    shuffle=True,
    pin_memory=True)

valid_dataloader = torch.utils.data.DataLoader(
    valset, batch_size=batch_size,
    shuffle=True,
    pin_memory=True)

test_dataloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size,
    shuffle=False,
    pin_memory=True)


# training

## starting the train loop

In [ ]:
for i in tqdm(range(epochs)):
    train_loss=0.


    for batchd in train_dataloader:
        model.train()
        noisy_img, org_img=batchd
       
        #shift batch dimension to the beginning according to documentation for Conv layer

        #print(type(noisy_img))

        noisy_img = noisy_img.to(device)
        org_img = org_img.to(device)
       
        noisy_img = torch.reshape(noisy_img,(noisy_img.shape[0],1,256))
        denoised_img=model(noisy_img)
        #denoised_img=denoised_img.permute(0,3,1,2)
        #loss=criterion(denoised_img, org_img) #tra quella aggiustata e quella originale senza noise
        loss=theloss(denoised_img, org_img,0)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    valid_loss = 0

    for inputs in valid_dataloader:
        with torch.no_grad():
            model.eval()

            noisy_img, org_img= inputs


            #print(type(noisy_img))

            noisy_img = noisy_img.to(device)
            org_img = org_img.to(device)

            noisy_img = torch.reshape(noisy_img,(noisy_img.shape[0],1,256))
            denoised_img=model(noisy_img)
            #denoised_img=denoised_img.permute(0,3,1,2)
            #loss=criterion(denoised_img, org_img) #tra quella aggiustata e quella originale senza noise
            vloss=theloss(denoised_img, org_img,0)

            valid_loss+=vloss.item()


    #print(epochs, train_loss/len(train_dataloader), valid_loss/len(valid_dataloader))
    print("Epochs {} TRAIN_loss {} and VALID_loss {}".format(i+1,train_loss/len(train_dataloader),valid_loss/len(valid_dataloader)))

## Test section

## model uploads (when needed). 
Here you can just run inference of the trained model, or upload a trained one and directly using it.

In [ ]:
#del(model)

# map_location=torch.device('cpu')
#model = torch.load('models/measurementSettingsModels/Dep0)TransformerDUEMeasurementSettingsA-MSE.pth',map_location=torch.device('cpu'))


## Inference

In [ ]:


reconstructions = []
originals = []
modelInputs = []

with torch.no_grad():

  for inputs in test_dataloader:
    model.eval()

    noisy_img, org_img= inputs

    noisy_img = noisy_img.to(device)
    org_img = org_img.to(device)
    noisy_img = torch.reshape(noisy_img,(noisy_img.shape[0],1,256))
    denoised_img=model(noisy_img)
    #print(noisy_img.shape)
    #step two

    for _ in range(3):
        denoised_img = model(torch.reshape(denoised_img,(10,1,256)))
      
    reconstructions.extend(denoised_img.cpu().detach())
    originals.extend(org_img.cpu().detach())
    modelInputs.extend(noisy_img.cpu().detach())


## density matricy trace value test 

In [ ]:
len(reconstructions)
unit = []
for el in reconstructions:
    unit.append(el[0])
np.mean(unit)

## state reconstruction (projection)

In [ ]:
#ulteriore postprocessing, ritorno a correlatori nella forma originale e proietto

#first off, generate the basis once!
pauliBasis = generatePaulimatrices()

fidNoisyTarget = []
fidProjectedTarget =[]
PurNoisyTarget = []
PurProjectedTarget =[]

def unnormalizeForAttention(inp):
    inp=inp.numpy()
    inp = 2*inp -1
    inp= delete(inp,0)
    return inp
i = 0
di = 100

def globalDepo(rm):
	p= 0.2
	i = np.eye(16,16)
	dep = (1-p)*rm + (p/16.)*i
	return dep
#pbasis = generatePaulimatrices()

for corretta, rumorosa, originale in zip(reconstructions[i:i+di], modelInputs[i:i+di], originals[i:i+di]):

    corretta_vec = unnormalizeForAttention(corretta)
    rumorosa_vec = unnormalizeForAttention(rumorosa)
    originale_vec = unnormalizeForAttention(originale)

    #print('rumoros',rumorosa_vec[:20].shape)
    #OLD RANDOM SET
    #corrs_counts='100101000010110000000000001010000000001100000000000000000000000000000000000000000010000000000010000000000100000000100000000000000000000110010000000000110000000001000000000000000000010000001000010000000000000000000010100000010000000101000000000010000000010'
 

    #several corrs_counts
    f = open('data/fromMeasurementSettings/'+foldertype+'/corrsCountList.txt', 'r')
    corrs_counts = f.read()
    print(len(corrs_counts))
    
    #2) PROIEZIONE
    tiponorma="fro"

    #print(corretta_vec)
    #nota, inserisco rumorosa_vec invece di originale vec
    corrttproj_dm=proj(corrs_counts, corretta_vec, rumorosa_vec,tiponorma,pauliBasis)[1]
    org_dm=stato_from_corr(originale_vec, pauliBasis)
    org_dm = globalDepo(org_dm)
    
    noisy_dm=stato_from_corr(rumorosa_vec, pauliBasis)


    print('PROJ PURiTY', np.trace(corrttproj_dm@corrttproj_dm)) 
    
    fidNoisyTarget.append(fidelity(Qobj(noisy_dm), Qobj(org_dm)))
    fidProjectedTarget.append(fidelity(Qobj(corrttproj_dm), Qobj(org_dm)))

    PurNoisyTarget.append((Qobj(noisy_dm)*Qobj(noisy_dm)).tr())
    PurProjectedTarget.append((Qobj(corrttproj_dm)*Qobj(corrttproj_dm)).tr() )



## TRACE NORM TEST

In [ ]:
def traceNormTest(m):
 
    e = np.linalg.eigvals(m)
    return sum(np.abs(e))
tt = []

for el in reconstructions:
    cc = unnormalizeForAttention(el)
    netOut = stato_from_corr(cc, pauliBasis)
    tt.append(traceNormTest(netOut))


## Plots, averaged values, savings

In [ ]:
np.mean(fidNoisyTarget), np.mean(fidProjectedTarget), np.mean(PurNoisyTarget), np.mean(PurProjectedTarget), len(fidNoisyTarget)

## SAVING

In [ ]:
plotfold = 'fromMeasurementSets/'+'B4/'
torch.save(reconstructions,'parallel/'+str(plotfold)+'/SetB4Transformer-reconstruction6k')
torch.save(originals,'parallel/'+str(plotfold)+'/SetB4Transformer-originals6k')
torch.save(modelInputs,'parallel/'+str(plotfold)+'/SetB4Transformer-modelInputs6k')

In [ ]:
torch.save(model,'models/measurementSettingsModels/Dep0TransformerDUEMeasurementSettings'+foldertype[0]+'-MSE.pth')


In [ ]:
import gc
del(model)
del(fidNoisyTarget)
del(fidProjectedTarget)
del(PurNoisyTarget)
del(PurProjectedTarget)
torch.cuda.empty_cache()
gc.collect()

In [ ]:
#randomcorrs_TEST='100101000010110000000000001010000000001100000000000000000000000000000000000000000010\
#    00000000001000000000010000000010000000000000000000011001000000000011000000000100000000000000000001000\
#    0001000010000000000000000000010100000010000000101000000000010000000010'



